In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
df = pd.read_csv("DF_BOOTCAMPS_2591.csv")

In [7]:
#CONSERVAR A TODOS AQUELLOS QUE NO TENGAN MISSING VALUES EN EDUCATION Y CERTIFICATION. 

dfa = df[df["about"].notna()] #DATAFRAME DE EDUCATION
dfa = dfa.reset_index(drop = True) #REINICIAR INDICE

In [8]:
dfa.columns

Index(['id', 'scraping_services_id', 'linkedin_id', 'linkedin_profile_url',
       'last_name', 'first_name', 'v_extension', 'location', 'job_title',
       'company', 'job_history', 'snapshots', 'skills', 'education', 'about',
       'updated', 'middle_name', 'num_connections', 'interests', 'volunteered',
       'certifications', 'languages', 'recommendations', 'publications',
       'patents', 'industry', 'titles', 'company_name_complete',
       'job_title_complete'],
      dtype='object')

In [9]:
#EL NUMERO DE NO NAS PARA DE 2500 A 1423
dfa.shape

(1423, 29)

In [10]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'dev f\\.'
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'reprograma',
 'worldtech', 
 'bictia|bi.?tia', 
 'talento[ -]?digital']

# EXTRAER FORMACIÓN DESDE EL DATASET DE LOS NO-MISSING VALUES DE ABOUT

In [12]:
#CONVERTIR EN STRING
dfa["about"] =  dfa["about"].str.lower()

In [22]:
lista_boots = ['4Geeks',
 'Acámica',
 'Ada ITW',
 'Bedu Tech',
 'Bogotá Dev',
 'Bootcamp Institute',
 'Coderise International',
 'Desafío Latam',
 'Dev F.',
 'Digital House',
 'Generation Mexico',
 'Hack Academy',
 'HolaCode',
 'Holberton School',
 'iCreate Coding Academy',
 'IJALTI',
 'Iron Hack',
 'Jóvenes a Programar',
 'Keep Coding',
 'Kod1go',
 'Laboratoria',
 'Le Wagon',
 'Make it Real Camp',
 'MindHub ',
 'Muktek Academy',
 'Nivel Pro',
 'Noukod',
 'Plataforma 5',
 'Programa Valentina',
 'Pygmalion',
 'Reprograma',
 'World Tech Inc (a.k.a. World Tech Makers –WTM–)',
 'BICTIA',
 'Talento Digital']

In [13]:
# REGEX DE BOOTCAMPS
boots =['4[ -]?geeks',
 'ac.mica',
 'ada.?itw',
 'bedu[ -]?tech', 
 'bogot.[ -]?dev| bogot.[ -]?bootcamp',
 'bootcamp[ -]?institute',
 'coderise|coderise.?international',
 'desaf.o[ -]?latam',
 'dev ?f\.',
 'digital[ -]?house',
 'generation[ -]?m.xico',
 'hack.?academy',
 'hola.?code',
 'holberton.?school',
 'icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding',
 'ijalti', 
 'iron.?hack',
 'j.venes[ -]?a[ -]?programar',
 'keep.?coding|keep.?coding.io',
 'kod[ -]?1[ -]?go',
 'laborat.ria ', 
 'le.?wagon', 
 'make[ -]?it[ -]?real[ -]?camp| make[ -]?it[ -]?real',
 'mind.?hub',
 'muktek|muk.?tek', 
 'nivel[ -]?pro ', 
 'noukod|noukood|noukd', 
 'plataforma[ -]?5',
 'programa[ -]?valentina',
 'pygmalion'
 'reprograma',
 'worldtech$',  #se cambió  para que solo termine con $
 'bictia|bi.?tia', 
 'talento[ -]?digital$'] #

In [32]:
#DADO QUE AL ABOUT NO TIENE NINGUNA ESTRUCTURA, SOLO RESTA BUSCAR EN LOS STRINGS DE CADA INDIVIDUO
c = []
for boot in range(len(boots)): #PARA CADA BOOTCAMP
    for i in range(len(dfa.about)): #PARA CADA INDIVIDUO
        A = re.findall(boots[boot], dfa.about[i])
        if len(A) > 0:
            c.append((dfa.id[i], boots[boot], lista_boots[boot]))
            print(f"{dfa['job_title'][i]}---{A[0]}---{boots[boot]}--- \n {dfa.about[i]} \n ------------------------------------------\n")


Company Director---icreate---icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding--- 
 hi welcome to my linkdn business page, i am looking for creative writers and poets to join my small book business icreatedreamz.biz limited asap to help me to organise and help writers who long to get their books published and to guide them with characters, plots, ideas and unique storylines. i am also looking for editors and proofreaders and formatters and self publishers all help will be gratefully recieved to keep my small book business sailing and not to sink thank you. 
 ------------------------------------------

Intermediate Web Developer---icreate---icreate[ -]?limited|icreate|icreate[ -]?coding[ -]?academy| icreate[ -]?coding--- 
 please visit my website -> http://www.jinawebdev.com/

 i'm a student who is studying web design and development at yoobee design school in wellington. i finished icreate course last year and taking a web development course in this year.



Parece que el desempeño en este caso es positivo. En los perfiles de about parece haber información relevante.

In [33]:
#CREAR DATAFRAME
c = list(set(c))
educ_boots = pd.DataFrame.from_records(c, columns=['id', 'regex_bootcamp', 'bootcamp'])

In [34]:
len(c)
#14 POTENCIALES PERFILES DESDE ABOUT

14

In [35]:
#HACER MERGE CON EL RESTO DE COLUMNAS
about_boots = pd.merge(educ_boots, dfa, "left", "id")
about_boots.to_excel("2_about_boots.xlsx", index = False)

In [36]:
about_boots

,id,regex_bootcamp,bootcamp,scraping_services_id,linkedin_id,linkedin_profile_url,last_name,first_name,v_extension,location,...,volunteered,certifications,languages,recommendations,publications,patents,industry,titles,company_name_complete,job_title_complete
0,49579953,keep.?coding|keep.?coding.io,Keep Coding,49579953,daviddetena,https://www.linkedin.com/in/daviddetena,Calvo,David,"{'gender': 'male', 'industry_from_job': False,...","{'country_name': 'Spain', 'state_name': 'Andal...",...,NaN,"[{'certificate_authority': 'KeepCoding', 'end'...",['Inglés'],NaN,[{'authors': [{'full_name': 'David de Tena Cal...,NaN,NaN,NaN,NaN,NaN
1,79419003,laborat.ria,Laboratoria,79419003,sthefanyfloriano,https://www.linkedin.com/in/sthefanyfloriano,Floriano,Sthefany,"{'gender': False, 'industry_from_job': 'Inform...",{'country_name': 'Peru'},...,{'0': {'description': 'La comunidad de la inno...,"[{'certificate_authority': 'CIBERTEC', 'end': ...","['Español', 'Inglés']",NaN,[{'authors': [{'full_name': 'Sthefany Floriano...,NaN,NaN,NaN,NaN,NaN
2,14259255,icreate[ -]?limited|icreate|icreate[ -]?coding...,iCreate Coding Academy,14259255,donna-o-ferrall-36b2b3126,https://www.linkedin.com/in/donna-o-ferrall-36...,Oferrall,Donna,"{'gender': 'female', 'industry_from_job': Fals...",{'country_name': 'United Kingdom'},...,"[{'causes': ['Animal Welfare', 'Arts and Cultu...","[{'end': 'Present', 'title': 'appointed Compan...",['English'],['Donna is very creative. She takes the first ...,"[{'authors': [{'full_name': ""Donna O'ferrall"",...",NaN,Writing & Editing,NaN,NaN,NaN
3,59233846,keep.?coding|keep.?coding.io,Keep Coding,59233846,ivanalexisabad,https://www.linkedin.com/in/ivanalexisabad,Abad,Ivan,"{'gender': 'male', 'industry_from_job': 'Infor...",{'country_name': 'Spain'},...,"[{'causes': ['Animal Welfare', 'Environment', ...",NaN,"['English', 'Spanish']","['Podcaster de corazón, gran locución, consigu...",NaN,NaN,NaN,NaN,NaN,NaN
4,50136139,laborat.ria,Laboratoria,50136139,kiara-esquerra-ferruzo-91588a120,https://www.linkedin.com/in/kiara-esquerra-fer...,Ferruzo,Kiara,"{'gender': 'female', 'industry_from_job': None...",{'country_name': 'Peru'},...,[{'description': 'Participé en un voluntariado...,[{'certificate_authority': 'University of Camb...,"['Español', 'Alemán', 'Francés', 'Inglés']",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,50491438,keep.?coding|keep.?coding.io,Keep Coding,50491438,jacoboenriquez,https://www.linkedin.com/in/jacoboenriquez,Gabeiras,Jacobo,"{'gender': 'male', 'industry_from_job': 'Compu...","{'country_name': 'Spain', 'state_name': 'Comun...",...,"[{'causes': ['Civil Rights and Social Action',...","[{'certificate_authority': 'Duolingo', 'end': ...","['Inglés', 'Francés', 'Español']","['Es una persona muy resolutiva, con amplios c...",NaN,NaN,NaN,NaN,NaN,NaN
6,50558792,laborat.ria,Laboratoria,50558792,evelyn-farfan,https://www.linkedin.com/in/evelyn-farfan,Farfan,Evelyn,"{'gender': 'female', 'industry_from_job': 'Des...",{'country_name': 'Peru'},...,"{'1': {'causes': ['Animal Welfare', 'Arts and ...",NaN,['Español'],NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,49553788,laborat.ria,Laboratoria,49553788,jenniferjara,https://www.linkedin.com/in/jenniferjara,Rojas,Jennifer,"{'gender': 'female', 'industry_from_job': 'Com...",{'country_name': 'Peru'},...,"[{'causes': ['Animal Welfare', 'Arts and Cultu...","[{'certificate_authority': 'Codecademy', 'end'...",['Inglés (hablado y escrito)'],['Jennifer es una persona muy responsable orie...,NaN,NaN,NaN,NaN,NaN,NaN
8,47818340,laborat.ria,Laboratoria,47818340,sofia-caballero-m,https://www.linkedin.com/in/sofia-caballero-m,Caballero,Sofia,"{'gender': 'female', 'industry_from_job': None...",{'country_name': 'Peru'},...,[{'description': 'Ayudar a niños a entender co...,"[{'certificate_authority': 'ICPNA', 'end': 'Pr...","['Español', 'Ingles']",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,67980737,laborat.ria,Laboratoria,67980737,sheylabre,https://www.linkedin.com/in/sheylabre,Sicha,Sheyla,"{'gender': False, 'industry_from_job': False, ...",{'